In [2]:
import tarfile
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from cmapPy.pandasGEXpress.parse import parse
from functools import reduce


In [3]:
cancer_celllines = {"brca" : "MCF7", "luad": "A549", "lusc" : "A549", "lihc" : "HEPG2", "prad" : "PC3", "coad" : "HT29"}
cmp_data = parse("C:\\Users\\areeba khan\\Documents\\UdS\\Master Thesis\\CMAP_Output\\coad\\ps_pert_cell.gctx")
metadata = cmp_data.row_metadata_df
cmap_scores = cmp_data.data_df
# cmapdrugs_scores
cell_line = cancer_celllines["prad"]
metadata.reset_index(drop = True, inplace = True)
mcf7_brd_df = metadata[metadata['pert_id'].str.contains('BRD') & metadata['cell_id'].str.contains(cell_line) & metadata['pert_type'].str.contains('trt_cp')]

cmap_scores.reset_index(inplace=True)
cmap_scores.columns = ["cid", "TAG"]

mcf7_scores = cmap_scores[cmap_scores['cid'].str.contains('BRD') & cmap_scores['cid'].str.contains(cell_line)]
# mcf7_scores['pert_id'] = mcf7_scores['cid'].str.split(':', expand=True)[0]
mcf7_scores.loc[:, 'pert_id'] = mcf7_scores['cid'].str.split(':', expand=True)[0]

merged_df = pd.merge(mcf7_scores, mcf7_brd_df[['pert_id', 'pert_iname', 'pert_type']], on='pert_id')

c:\Users\areeba khan\AppData\Local\Programs\Python\Python310\lib\site-packages\cmapPy\pandasGEXpress\parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
c:\Users\areeba khan\AppData\Local\Programs\Python\Python310\lib\site-packages\cmapPy\pandasGEXpress\parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
C:\Users\areeba khan\AppData\Local\Temp\ipykernel_14080\1679234240.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [6]:
merged_df["pert_iname"].to_csv("C:\\Users\\areeba khan\\Documents\\UdS\\Master Thesis\\CMAP_Output\\all_prad\\prad_cmap_cmps.txt", index = False, header = False)

In [27]:
final_df = merged_df[['pert_id', 'pert_iname', 'TAG']]
cmap_name_cids = pd.read_csv(f"C:/Users/areeba khan/Documents/UdS/Master Thesis/CMAP_Output/brca/brca_cmap_cmps_cid.txt", header=None, sep="\t")

cmap_name_cids.columns = ['pert_iname', 'cid']

cmap_name_cids['pert_iname'] = cmap_name_cids['pert_iname'].str.lower()

cmap_name_cids.drop_duplicates(inplace=True)

cmap_name_cids = cmap_name_cids.dropna(subset=['cid'])

cmap_name_cids = cmap_name_cids.groupby('pert_iname')['cid'].agg(lambda x: ','.join(x.astype(str).unique())).reset_index()
cmap_name_cids['pert_iname'] = cmap_name_cids['pert_iname'].str.lower()

final_df['pert_iname'] = final_df['pert_iname'].str.lower()
cmap = pd.merge(cmap_name_cids, final_df[['pert_id', 'pert_iname', 'TAG']], on='pert_iname')
print(cmap)


                            pert_iname         cid        pert_id        TAG
0     1,2,3,4,5,6-hexabromocyclohexane     74603.0  BRD-K06817181 -19.991278
1                  1,2-dichlorobenzene      7239.0  BRD-K74430258   0.000000
2                    1-benzylimidazole     77918.0  BRD-K32795028  61.190712
3                       1-monopalmitin     14900.0  BRD-A80928489 -40.814514
4                    1-phenylbiguanide      4780.0  BRD-K31491153  52.198452
...                                ...         ...            ...        ...
2453                        zonisamide      5734.0  BRD-K48300629   1.236632
2454                        zosuquidar   3036703.0  BRD-K70557564   0.000000
2455                      zoxazolamine      6103.0  BRD-K66353228  22.357698
2456                          zstk-474  11647372.0  BRD-K63068307 -94.824883
2457                    zuclopenthixol   5311507.0  BRD-K28761384  52.662762

[2458 rows x 4 columns]


C:\Users\areeba khan\AppData\Local\Temp\ipykernel_14592\1018334190.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['pert_iname'] = final_df['pert_iname'].str.lower()


In [28]:
cmap['pert_iname'] = cmap['pert_iname'].str.lower()
cmap_grouped = cmap.groupby(['pert_iname', 'TAG'], as_index=False).agg({
    'cid': lambda x: ','.join(x)
})

cmap = cmap_grouped.drop_duplicates()

def unique_join(val):
    cids = val.split(',')
    cids = [str(int(float(cid))) for cid in cids]
    return ','.join(sorted(set(cids)))

cmap['cid'] = cmap['cid'].apply(unique_join)

cmap.reset_index(drop=True, inplace=True)

,pert_iname,TAG,cid
0,"1,2,3,4,5,6-hexabromocyclohexane",-19.991278,74603
1,"1,2-dichlorobenzene",0.000000,7239
2,1-benzylimidazole,61.190712,77918
3,1-monopalmitin,-40.814514,14900
4,1-phenylbiguanide,52.198452,4780
...,...,...,...
2438,zonisamide,1.236632,5734
2439,zosuquidar,0.000000,3036703
2440,zoxazolamine,22.357698,6103
2441,zstk-474,-94.824883,11647372


In [26]:
print(cmap)

                            pert_iname        TAG        pert_id       cid
0     1,2,3,4,5,6-hexabromocyclohexane -19.991278  BRD-K06817181     74603
1                  1,2-dichlorobenzene   0.000000  BRD-K74430258      7239
2                    1-benzylimidazole  61.190712  BRD-K32795028     77918
3                       1-monopalmitin -40.814514  BRD-A80928489     14900
4                    1-phenylbiguanide  52.198452  BRD-K31491153      4780
...                                ...        ...            ...       ...
2453                        zonisamide   1.236632  BRD-K48300629      5734
2454                        zosuquidar   0.000000  BRD-K70557564   3036703
2455                      zoxazolamine  22.357698  BRD-K66353228      6103
2456                          zstk-474 -94.824883  BRD-K63068307  11647372
2457                    zuclopenthixol  52.662762  BRD-K28761384   5311507

[2458 rows x 4 columns]


In [30]:
import json

# Define the path to the file that contains the data
input_file_path = "C:\\Users\\areeba khan\\Documents\\UdS\\Master Thesis\\CMAP_Output\\brca\\equal_set\\degs\\overlap_compounds.txt"

# Open the file and read the data
with open(input_file_path, 'r') as file:
    # Load the data from the file (assuming it's JSON-like format)
    data = json.load(file)

# Extract cmap_compounds and pert_id into a list of tuples (cmap_compounds, pert_id)
cmap_pert_list = [(entry['cmap_compunds'], entry['pert_id']) for entry in data]

# Print the list to see the output
print(cmap_pert_list)

[('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-A83237092'), ('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-A83237092'), ('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-A83237092'), ('doxorubicin', 'BRD-K92093830'), ('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-A83237092'), ('doxorubicin', 'BRD-K92093830'), ('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-A83237092'), ('doxorubicin', 'BRD-K92093830'), ('mitomycin-c', 'BRD-A48237631'), ('tamoxifen', 'BRD-K04210847'), ('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-A83237092'), ('doxorubicin', 'BRD-K92093830'), ('mitomycin-c', 'BRD-A48237631'), ('tamoxifen', 'BRD-K04210847'), ('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-A83237092'), ('doxorubicin', 'BRD-K92093830'), ('mitomycin-c', 'BRD-A48237631'), ('tamoxifen', 'BRD-K04210847'), ('doxorubicin', 'BRD-A52530684'), ('tamoxifen', 'BRD-K43744935'), ('fulvestrant', 'BRD-A90490067'), ('palbociclib', 'BRD-K51313569'), ('fulvestrant', 'BRD-